In [1]:
import pandas as pd
import tensorflow as tf

In [6]:
data = pd.read_csv('/content/drive/MyDrive/Datascience/Fake News/train.csv')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
data = data.dropna()

In [8]:
X = data.drop('label', axis =1) 
y = data['label']

In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
#Vocabulary size
voc_size = 5000

In [13]:
#Onehot Representation
messages = X.copy()
messages.reset_index(inplace = True)

In [14]:
#Data preprocessing
import nltk
import re
from nltk.corpus import stopwords


In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [17]:
corpus[0:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [18]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]
onehot_rep

[[3085, 4474, 4491, 4829, 1745, 2591, 4268, 4242, 1036, 2075],
 [1512, 4079, 4067, 2490, 3965, 267, 329],
 [3658, 1771, 2937, 1946],
 [9, 4355, 253, 4810, 2120, 1119],
 [1344, 3965, 4168, 3076, 3612, 4710, 3965, 4902, 3794, 4751],
 [811,
  323,
  3233,
  3839,
  1495,
  1072,
  993,
  4535,
  3636,
  15,
  3159,
  2285,
  2120,
  1715,
  329],
 [3210, 1057, 193, 2740, 1337, 3252, 1811, 1439, 1074, 1587, 2195],
 [3795, 419, 4545, 1150, 4839, 2870, 1072, 1740, 1074, 1587, 2195],
 [2684, 4714, 4868, 1292, 2645, 4203, 2425, 4872, 1072, 4702],
 [3420, 1304, 4156, 4409, 2198, 1832, 831, 310],
 [3727, 3826, 4304, 3210, 4010, 3498, 4553, 608, 469, 2816, 767],
 [4810, 1830, 1745, 4203, 1072, 4839],
 [4725, 3019, 1247, 2128, 4219, 247, 4835, 1322, 1490],
 [1018, 2739, 1043, 4530, 2687, 1405, 3334, 1074, 1587, 2195],
 [4176, 1580, 3071, 3889, 1802, 1074, 1587, 2195],
 [2017, 2199, 1432, 2644, 519, 1547, 185, 3912, 157, 4092],
 [3469, 2221, 4079],
 [2753, 1981, 791, 4533, 1072, 2671, 3359, 329],
 

In [19]:
#Padding sequences
sent_length = 20
embedded_doc = pad_sequences(onehot_rep, padding = 'pre', maxlen = sent_length)



In [20]:
embedded_doc

array([[   0,    0,    0, ..., 4242, 1036, 2075],
       [   0,    0,    0, ..., 3965,  267,  329],
       [   0,    0,    0, ..., 1771, 2937, 1946],
       ...,
       [   0,    0,    0, ..., 1074, 1587, 2195],
       [   0,    0,    0, ..., 1450,  239, 2698],
       [   0,    0,    0, ..., 3232,  884, 4958]], dtype=int32)

In [23]:
embedding_vectorfeatures = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vectorfeatures, input_length= sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss= 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [25]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [26]:
import numpy as np
X_final = np.array(embedded_doc)
y_final = np.array(y)

In [27]:
X_final.shape

(18285, 20)

In [28]:
y_final.shape

(18285,)

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.3, random_state = 42)

In [32]:
model.fit(X_train,y_train, validation_data = (X_test,y_test), epochs = 10, batch_size = 64)

Epoch 1/10
200/200 [==============================] - 6s 32ms/step - loss: 0.1169 - accuracy: 0.9569 - val_loss: 0.1287 - val_accuracy: 0.9539
Epoch 2/10
200/200 [==============================] - 6s 32ms/step - loss: 0.0823 - accuracy: 0.9716 - val_loss: 0.1123 - val_accuracy: 0.9626
Epoch 3/10
200/200 [==============================] - 7s 33ms/step - loss: 0.0602 - accuracy: 0.9806 - val_loss: 0.1078 - val_accuracy: 0.9696
Epoch 4/10
200/200 [==============================] - 7s 33ms/step - loss: 0.0409 - accuracy: 0.9873 - val_loss: 0.0943 - val_accuracy: 0.9730
Epoch 5/10
200/200 [==============================] - 7s 33ms/step - loss: 0.0255 - accuracy: 0.9923 - val_loss: 0.1105 - val_accuracy: 0.9736
Epoch 6/10
200/200 [==============================] - 7s 34ms/step - loss: 0.0190 - accuracy: 0.9942 - val_loss: 0.1090 - val_accuracy: 0.9741
Epoch 7/10
200/200 [==============================] - 7s 33ms/step - loss: 0.0131 - accuracy: 0.9962 - val_loss: 0.1067 - val_accuracy: 0.9772

In [34]:
y_pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [36]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cm

array([[2839,  268],
       [ 114, 2265]])

In [37]:
score = accuracy_score(y_test, y_pred)
score

0.9303682099890631